# 1. Radar
Por: Camila Lopes ([cclopes.me](https://cclopes.me/))  


Neste módulo, iremos aprender a lidar com dados de radares meteorológicos usando a linguagem Python, passando por leitura, visualização e processamento.


## 1.1. Lendo dados de radar

### 1.1.1. Tipos de dados
Antes de aprender a ler dados de radar, é importante entender quais são os tipos de dados gerados por radares, já que eles podem ser configurados de diversas formas.

#### Escaneamento
A partir da fabricação do radar, é definido em qual banda ele irá operar e qual será sua configuração de escaneamento. Além de depender da banda de operação, essa configuração vai depender do local em que o radar será instalado e do objetivo dele (monitoramento ou pesquisa).  

Radares de monitoramento são de bandas de frequência menor (2 a 8 GHz, Bandas S e C) pois sofrem menos atenuação. Para tirar mais vantagem ainda disso, suas configurações de escaneamento são do tipo **volumétricas**, ou seja, em função do azimute (plano horizontal) e elevação (altura). A resolução temporal varia bastante dependendo principalmente da quantidade de elevações: dependendo do local a ser monitorado, uma quantidade menor ou maior de elevações é feita, com o objetivo final de ter uma resolução temporal razoável para monitoramento em tempo real (em geral essa resolução varia entre 5 e 15 minutos). Os dados gerados por esse tipo de configuração são no formato **PPI** (*Plan Position Indicator*), equivalente a uma matriz tridimensional mas não cartesiana por ser em função do azimute e elevação. Visualizações desses PPIs podem ser feitas no "plano horizontal" (imagem de todos os azimutes em uma elevação fixa) ou "vertical" (imagem de todas as elevações em um azimute fixo). Para gerar produtos mais simples de serem interpretados, o formato PPI é convertido para **CAPPI** (*Constant Altitude Plan Position Indicator*), agora sim equivalente a uma matriz tridimensional cartesiana (plano horizontal e altura). As visualizações dos CAPPIs podem ser feitas no plano horizontal (altura fixa) ou vertical (eixo x ou y fixo e altura variada).  

Radares de pesquisa costumam ser de frequência maior (acima de 8 GHz, Bandas X, K e W) pois, mesmo sofrendo mais atenuação principalmente de gotas de chuva, são mais sensíveis à gotículas de nuvem e cristais de gelo, permitindo estudos de microfísica de nuvens. Além do escaneamento volumétrico, uma configuração interessante para esses estudos é do tipo **setorial**, em que uma região de azimutes é fixada e o escaneamento é feito em função da elevação para cada azimute. Por medir apenas um setor, a resolução temporal pode ser menor que 5 minutos. Os dados gerados por esse tipo de configuração são no formato **RHI** (*Range Height Indicator*), equivalente a uma matriz tridimensional mas não cartesiana por ser em função da elevação e azimute. A visualização de RHIs em geral são feitas no "plano vertical" (distância em função da altura), mas também podem ser feitas no "plano horizontal" (todos os azimutes disponíveis em função de uma elevação).  

Os radares de pesquisa das bandas K e W (acima de 20 GHz) sofrem ainda mais atenuação, de forma que outra configuração de escaneamento que é usada (dependendo da fabricação do radar, é a única configuração disponível) é do tipo **apontamento vertical**, em que, como o nome já diz, a antena fica apontada para cima (ou para baixo, se estiver a bordo de um avião ou satélite), medindo uma mesma região ao longo do tempo. A resolução temporal nessa configuração é na escala de segundos a poucos minutos. Os dados gerados por esse tipo de configuração são no formato **THI** (*Time Height Indicator*), equivalente a uma matriz bidimensional em função da altura e tempo, com a visualização nesse mesmo plano.

#### Formato
Existem diferentes formatos de arquivos que armazenam os tipos de dados explicados acima. Os principais são:

- **Binário** (extensão `.bin` ou sem extensão): este é o formato mais simples de armazenamento. Ele pode ter diferentes tipos de estruturas (com cabeçalho ou não), mas não é possível saber essa estrutura apenas com o arquivo - é preciso ter algum README associado para conseguir abrir o arquivo. Sabendo essa estrutura, é possível abrir esse tipo de arquivo com a função `open` do Python. Como exemplo [baseado neste artigo do Towards Data Science](https://towardsdatascience.com/loading-binary-data-to-numpy-pandas-9caa03eb0672), supondo que a estrutura do arquivo `filename.bin` é simplesmente uma matriz de um nível de CAPPI de 500 x 500 pontos, em que cada ponto é um `integer` de 4 bytes, o código abaixo lê esse arquivo e converte para uma matriz do `numpy`:  

```
# Carregando a biblioteca
import numpy as np

# Definindo o formato da matriz
dt = np.dtype([('x', '<i4'), ('y', '<i4')])

# Abrindo o arquivo
with open("filename.bin", "rb") as f:
    binary_data = f.read()

# Convertendo para a matriz
np_data = np.frombuffer(binary_data, dt)
```

Esse tipo de arquivo ainda pode ser encontrado para dados de radar brutos, mas não para dados pré-processados ou processados por terem estruturas cada vez mais complexas e a necessidade de armazenar mais informações além dos dados em si; 

- **uf** (*Universal Format*, `.uf`): este formato foi desenvolvido nos anos 1970 e era o mais comum para armazenamento e distribuição de dados de radar. Ele foi desenvolvido para ser processado usando a linguagem FORTRAN e possui uma estrutura pré-definida com cabeçalho em caracteres ASCII e conteúdo em `integers` de 16 bits (saiba mais sobre ela no [documento do TITAN sobre o formato uf](https://ral.ucar.edu/projects/titan/docs/radial_formats/UfDoc.txt)). Como a estrutura é conhecida, é possível abrir esse tipo de arquivo no Python da mesma forma que dados binários, mas algumas bibliotecas como [`pyart`](https://arm-doe.github.io/pyart/index.html) possuem [funções que leem arquivos uf](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_uf.html?highlight=read%20uf);

- **HDF5** (*Hierarchical Data Format*, `.HDF5`): este formato é utilizado para armazenar grande quantidade de dados de quaisquer dimensões de forma mais organizada, com metadados associados. Diferente dos dados binários, não é preciso saber a estrutura dos dados. Em Python, é necessário usar a função `File()` da biblioteca [`h5py`](https://docs.h5py.org/en/stable/) para abrir esse tipo de arquivo. Algumas bibliotecas que processam dados no formato HDF5 usam essa biblioteca dentro de suas funções;

- **NetCDF** (*Network Common Data Form*, `.nc`): este formato foi desenvolvido nos anos 1980 pela [UCAR (*University Corporation for Atmospheric Research*)](https://www.ucar.edu/) especificamente para armazenar e compartilhar dados científicos. Atualmente na versão 4 [mantida pela Unidata](https://www.unidata.ucar.edu/software/netcdf/), ele possui suporte em Java, FORTRAN, C e diversas outras linguagens de programação. Similar ao HDF5, o formato NetCDF armazena dados em quaisquer dimensões com metadados. Existe uma convenção para metadados de meteorologia chamada [CF (*Climate and Forecast*)](http://cfconventions.org/) com o objetivo de padronizar e facilitar o processamento de dados de diversas fontes; para dados de radar, essa convenção é chamada [CfRadial](https://github.com/NCAR/CfRadial). Em Python, é possível abrir esse tipo de arquivo diretamente com a biblioteca [`netCDF4`](https://unidata.github.io/netcdf4-python/netCDF4/index.html), mas diversas bibliotecas usam essa como base para processar dados NetCDF, atualmente sendo recomendável a biblioteca [`xarray`](http://xarray.pydata.org/en/stable/weather-climate.html);

- Outros: outros formatos de arquivo estão associados aos softwares dos radares que geraram os dados. As bibliotecas [`pyart`](https://arm-doe.github.io/pyart/index.html) e [`wradlib`](https://docs.wradlib.org/en/stable/index.html) que veremos a seguir tem suporte a diversos formatos como [**MDV**](https://ral.ucar.edu/projects/titan/docs/MDV_XML_ICD.1.0.pdf), [**IRIS** (Vaisala)](ftp://ftp.sigmet.com/outgoing/manuals/), [**GAMIC**](https://www.gamic.com/), [**RAINBOW**](https://www.leonardocompany.com/en/products/rainbow-5-application-software), entre outros.  

Sabendo agora quais são os tipos de dados, podemos aprender como ler dados de radar com as bibliotecas Py-ART e wradlib.

### 1.1.2. Py-ART

### 1.1.3. wrablib

## 1.2. Visualizando dados de radar

### 1.2.1. Py-ART

### 1.2.2. wradlib

## 1.3. Processando dados de radar

### 1.3.1. Py-ART

### 1.3.2. Py-ART + outras bibliotecas

### 1.3.3. wradlib